# IKEv2 responder test

In [ ]:
!apt update && apt install -y libpcap-dev
%pip install --pre scapy[basic]
%pip install pycryptodomex

In [ ]:
import socket
hostip = socket.gethostbyname(socket.gethostname())
print("IP address of host:", hostip)

In [ ]:
%load_ext autoreload
%autoreload

from scapy.all import *
from scapy.contrib.ikev2 import *
from scapy.utils import inet_aton
import binascii
import os
import ikev2_lib
import socket

IKEv2 specification: https://tools.ietf.org/pdf/rfc7296.pdf

In [ ]:
INITIATOR_IP = '172.17.0.3'
INITIATOR_PORT = 500
RESPONDER_IP = '172.17.0.2'
RESPONDER_PORT = 500

### Create dummy UDP listener to prevent ICMP port unreachable messages

In [ ]:
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind((RESPONDER_IP, RESPONDER_PORT))

### Initialise local parameters and secrets

In [ ]:
nonce_r = os.urandom(32)
spi_r = b'Ysblokje'

dh = ikev2_lib.DiffieHellman()

In [ ]:
capture = sniff(filter=f"dst host {RESPONDER_IP} and dst port {RESPONDER_PORT}", count=1)
capture.summary()

In [ ]:
packet = capture[0][IP]
packet.show()

In [ ]:
dh_a = dh.generate_public().to_bytes(length=256, byteorder='big') 
dh_b = int.from_bytes(packet[IKEv2_payload_KE].load, byteorder='big')

dhs = dh.generate_shared(dh_b).to_bytes(length=256, byteorder='big')
print(f"shared Diffie Hellman secret: {binascii.b2a_hex(dhs).decode()}")

In [ ]:
nonce_i = packet[IKEv2_payload_Nonce].load
spi_i = packet[IKEv2].init_SPI

print(f"SPIi: {spi_i}")
print(f"SPIr: {spi_r}")

print(f"Ni: {binascii.b2a_hex(nonce_i).decode()}")
print(f"Nr: {binascii.b2a_hex(nonce_r).decode()}")

### Calculate SKEYSEED and derive keys ([RFC7296 2.14](https://datatracker.ietf.org/doc/html/rfc7296#section-2.14))

In [ ]:
skeyseed = ikev2_lib.PrfHmacSha256(nonce_i + nonce_r, dhs)

print(f"SKEYSEED: {binascii.b2a_hex(skeyseed).decode()}")

Keys needed: SK_d | SK_ai | SK_ar | SK_ei | SK_er | SK_pi | SK_pr
* Sk_d is 32 bytes (256 bits) based on PRF_HMAC_SHA2_256
* Sk_ai is 20 bytes (160 bits) based on AUTH_HMAC_SHA1_96
* Sk_ar is 20 bytes (160 bits) based on AUTH_HMAC_SHA1_96
* Sk_ei is 32 bytes (256 bits) based on ENCR_AES_CBC
* Sk_er is 32 bytes (256 bits) based on ENCR_AES_CBC
* Sk_pi is 32 bytes (256 bits) based on PRF_HMAC_SHA2_256
* Sk_pr is 32 bytes (256 bits) based on PRF_HMAC_SHA2_256

Total is **200 bytes**

In [ ]:
prfplusoutput = ikev2_lib.PrfPlus(ikev2_lib.PrfHmacSha256, skeyseed, nonce_i + nonce_r + spi_i + spi_r, 200)
#print(f"prf+ output: {binascii.b2a_hex(prfplusoutput).decode()}")
sk_d = prfplusoutput[:32]
sk_ai = prfplusoutput[32:52]
sk_ar = prfplusoutput[52:72]
sk_ei = prfplusoutput[72:104]
sk_er = prfplusoutput[104:136]
sk_pi = prfplusoutput[136:168]
sk_pr = prfplusoutput[168:200]
print(f"Sk_d: {binascii.b2a_hex(sk_d).decode()}")
print(f"Sk_ai: {binascii.b2a_hex(sk_ai).decode()}")
print(f"Sk_ar: {binascii.b2a_hex(sk_ar).decode()}")
print(f"Sk_ei: {binascii.b2a_hex(sk_ei).decode()}")
print(f"Sk_er: {binascii.b2a_hex(sk_er).decode()}")
print(f"Sk_pi: {binascii.b2a_hex(sk_pi).decode()}")
print(f"Sk_pr: {binascii.b2a_hex(sk_pr).decode()}")

### Build IKE_SA_INIT response

In [ ]:
# Get proposal from Initiator IKE_SA_INIT 
proposal = packet[IKEv2_payload_Proposal]

hdr = IKEv2(init_SPI = spi_i, resp_SPI=spi_r, next_payload = 'SA', exch_type = 'IKE_SA_INIT', flags='Response')
sa = IKEv2_payload_SA(next_payload = 'KE', prop=proposal)
ke = IKEv2_payload_KE(next_payload = 'Nonce', group = '2048MODPgr', load = dh_a)
nonce = IKEv2_payload_Nonce(next_payload = 'None', load = nonce_r)

ike_sa_init = hdr/sa/ke/nonce

packet = IP(dst = INITIATOR_IP)/UDP(dport = INITIATOR_PORT, sport = RESPONDER_PORT)/ike_sa_init

#packet.show()


### Send IKE_SA_INIT response and receive IKE_AUTH

In [ ]:
ans = sr1(packet)
ans.show()

### Verify checksum and decrypt payload ([RFC7296 3.14](https://datatracker.ietf.org/doc/html/rfc7296#section-3.14))

In [ ]:
from Cryptodome.Cipher import AES
cipher_block_size = ikev2_lib.encryption[12]["block_size"] # AES_CBC
integrity_hash_size = ikev2_lib.integrity[2]["hash_size"] # HMAC_SHA1_96

auth_data = raw(ans[IKEv2])[:-integrity_hash_size]
iv = ans[IKEv2_payload_Encrypted].load[:cipher_block_size]
encrypted = ans[IKEv2_payload_Encrypted].load[cipher_block_size:-integrity_hash_size]
checksum = ans[IKEv2_payload_Encrypted].load[-integrity_hash_size:]

print(f"IV: {binascii.b2a_hex(iv).decode()}")
print(f"Checksum in packet: \t{binascii.b2a_hex(checksum).decode()}\tOK? {ikev2_lib.verify_integrity(sk_ai, auth_data, checksum)}\n")

#TODO: stop in case of checksum failure

plain = ikev2_lib.decrypt_message(sk_ei, encrypted, iv)
payload = IKEv2_payload_IDi(plain) # Cast decrypted payload to IKEv2 payload(s), for now assumes IDi payload
payload.show()